# Coursera Applied Data Science Capstone - Week 3

## Part 1

First we import the necessary packages. We import 'BeautifulSoup' to scrape and parse our data, 'requests' to be able to retrieve the data on Wikipedia, 'pandas' to put our scraped data into a dataframe, and 'numpy' to perform manipulation on our strings.

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

We retrieve the html text from the wiki page, and put it into a "soup" object as 'lxml' using BeautifulSoup

In [6]:
soup = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text, 'lxml')

Looking through the source code shows us that all the text we need is within 'td' html. We assign all the 'td' sections to a "table_data" object. Then we create an empty list called "postcodes", and for every entry in "table_data" up to the 865th entry, we return the text within the 'td' html, and add it to the "postcodes" list. I admittedly found it to be the first 865 entries through brute force.

In [7]:
table_data = soup.find_all('td')
postcodes=[]
for i in table_data[0:864]:
    postcodes.append(i.text)

In our "postcodes" list, every 1st entry is a postal code, every 2nd entry is a borough, and every 3rd entry is a neighborhood. We create a dataframe to contain these, and slice it accordingly so that each column contains the correct data. This lists the columns alphabetically by default, so we need an extra line of code to make sure the columns are in the order we want.

In [41]:
df = pd.DataFrame({'Postal Code': postcodes[0::3], 'Borough': postcodes[1::3], 'Neighborhood': postcodes[2::3]})
df = df[['Postal Code','Borough','Neighborhood']]
df.head(12)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


We don't want the '\n' in our Neighborhood columns, so we delete them (by replacing them with nothing). We also change 'Not assigned' boroughs to NaN so that we can then delete the rows containing them.

In [42]:
df.Neighborhood = df.Neighborhood.str.replace('\n','')
df.Borough = df.Borough.replace('Not assigned', np.nan)
df = df.dropna()
df.head(12)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Now let's combine rows where the postal codes are the same.

In [43]:
df = df.groupby(['Postal Code','Borough'], as_index=False, sort=False).agg(','.join)
df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


Finally, if a neighborhood is 'Not assigned', we replace it with its respective borough name.

In [44]:
df.Neighborhood.replace('Not assigned', df.Borough, inplace=True)
df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


How many rows and columns does our dataframe have?

In [45]:
df.shape

(103, 3)

## Part 2

We use 'pandas' to read the csv from the url given on Coursera, and have a look at the csv dataframe.

In [19]:
geo_co = pd.read_csv('https://cocl.us/Geospatial_data')

In [46]:
geo_co.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's merge the 'geo_co' and 'df' dataframes.

In [47]:
df = pd.merge(df, geo_co, on='Postal Code')
df.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
